In [1]:
#Installing Standard Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
import nltk
import re
from nltk.corpus import stopwords
ps = nltk.PorterStemmer()

# Installing Libraries

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Checking CountVectorizer

In [3]:
#Dimensional analysis of countvectorizer
cv = CountVectorizer()

corpus = ["This program is created by Nikhil Jain",
         "Nikhil has created this program",
         "This program has many applications"]

X = cv.fit(corpus)
print(X.vocabulary_)
print(cv.get_feature_names())

X = cv.transform(corpus)
#X = cv.fit_transform(corpus) #combinng fit and transform in one step
print(X.shape)
print(X.toarray())
print(X)

df_demo_cv = pd.DataFrame(X.toarray(), columns = cv.get_feature_names())
print(df_demo_cv)

{'this': 9, 'program': 8, 'is': 4, 'created': 2, 'by': 1, 'nikhil': 7, 'jain': 5, 'has': 3, 'many': 6, 'applications': 0}
['applications', 'by', 'created', 'has', 'is', 'jain', 'many', 'nikhil', 'program', 'this']
(3, 10)
[[0 1 1 0 1 1 0 1 1 1]
 [0 0 1 1 0 0 0 1 1 1]
 [1 0 0 1 0 0 1 0 1 1]]
  (0, 1)	1
  (0, 2)	1
  (0, 4)	1
  (0, 5)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (1, 2)	1
  (1, 3)	1
  (1, 7)	1
  (1, 8)	1
  (1, 9)	1
  (2, 0)	1
  (2, 3)	1
  (2, 6)	1
  (2, 8)	1
  (2, 9)	1
   applications  by  created  has  is  jain  many  nikhil  program  this
0             0   1        1    0   1     1     0       1        1     1
1             0   0        1    1   0     0     0       1        1     1
2             1   0        0    1   0     0     1       0        1     1


# Custom Clean Function

In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')
stopword.append('label')

pd.options.mode.chained_assignment = None  # default='warn'


#custiom clean function
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', ' ', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text

[nltk_data] Downloading package stopwords to C:\Users\Nikhil
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
tweets_all = pd.read_csv("tweets_all_csv.csv") 

In [9]:
tweets_less = tweets_all[25000:35000]
tweets_less

,Unnamed: 0,label,tweet_id,date_time,user_id,tweet
25000,795000,0,2327193206,Thu Jun 25 08:02:16 PDT 2009,djcampos,Blah 5am still up daang I got deep problems
25001,795001,0,2327193455,Thu Jun 25 08:02:17 PDT 2009,RKF,@jenspeedy I would suggest avoiding 360 Living. Not goodness Try contacting Scott at MKCC mk...
25002,795002,0,2327193641,Thu Jun 25 08:02:18 PDT 2009,AnaHertz,@alexbroun I didn't convince myself I was fat and ugly someone else did a pretty good job of tha...
25003,795003,0,2327193806,Thu Jun 25 08:02:18 PDT 2009,yenafer,"@spotzle @jstarrh check on sunscreen, snacks, towels, suits, kids drinks, bags and tape for boy..."
25004,795004,0,2327193864,Thu Jun 25 08:02:19 PDT 2009,eppoponotumus,im sitting alone at TTE myself without my two michigan sisters for the first time in a decade.
...,...,...,...,...,...,...
34995,804995,1,1468599688,Tue Apr 07 02:39:04 PDT 2009,joscelinyeo,@ickleoriental hahahha.. U obviously don't hv one!! But maybe u can give me advice? Fdw.. Forei...
34996,804996,1,1468599702,Tue Apr 07 02:39:04 PDT 2009,serengetisunset,"@juliekoh It's an internet term, but it's spilled over into common use, in real life"
34997,804997,1,1468599765,Tue Apr 07 02:39:06 PDT 2009,broombeck,new day.... NEW TRACK!!!!
34998,804998,1,1468599793,Tue Apr 07 02:39:07 PDT 2009,aptronym,@foodieguide Okay we need to have a competition to see whether you or I have the dodgiest Canton...


In [10]:
tweets_less['label'].head

<bound method NDFrame.head of 25000    0
25001    0
25002    0
25003    0
25004    0
        ..
34995    1
34996    1
34997    1
34998    1
34999    1
Name: label, Length: 10000, dtype: int64>

In [11]:
tweets_less['label'].value_counts()

1    5001
0    4999
Name: label, dtype: int64

# Feature Names for CV

In [12]:
cv1 = CountVectorizer(analyzer = clean_text)

X = cv1.fit_transform(tweets_less['tweet'])
print(X.shape) 
#so for example 10 rows, having 69 unique tokens, or features, or columns, or words

(10000, 15989)


In [13]:
print(cv1.get_feature_names())

['', 'aaaaag', 'aaaaahhh', 'aaaaahhhh', 'aaaall', 'aaaand', 'aaaawwwhey', 'aaah', 'aaajaayradd', 'aaargh', 'aaarrrrgghhhh', 'aaaww', 'aaawwww', 'aad', 'aahoogendoorn', 'aaminahh', 'aampj', 'aargh', 'aaronfilmmak', 'aaronmiciah', 'aaronshal', 'aaru', 'abacab', 'abay', 'abbi', 'abbyjeann', 'abc', 'abcdefglyni', 'abcess', 'abdomen', 'abegrub', 'aber', 'aberdeenfc', 'aberystwythart', 'abhisheknagar', 'abhishektiwari', 'abhishekupadhya', 'abi', 'abigail', 'abigaillovesy', 'abik', 'abil', 'abipip', 'abisignorelli', 'abit', 'abl', 'ablt', 'abolish', 'aboogi', 'abroad', 'absolut', 'absolutley', 'absorb', 'abt', 'abus', 'ac', 'acabo', 'academi', 'accent', 'accept', 'access', 'accessquot', 'accid', 'accident', 'accom', 'accomod', 'accompani', 'accomplish', 'accord', 'account', 'accountlook', 'accross', 'accshan', 'acct', 'accur', 'accus', 'ace', 'acebowman', 'ach', 'achaha', 'achen', 'achetã', 'achiev', 'achievin', 'achil', 'achm', 'ack', 'ackin', 'acknowledg', 'acland', 'acm', 'acmast', 'acowbo

In [14]:
df2 = pd.DataFrame(X.toarray(), columns = cv1.get_feature_names())
df2.head(10)

,,aaaaag,aaaaahhh,aaaaahhhh,aaaall,aaaand,aaaawwwhey,aaah,aaajaayradd,aaargh,...,á,â,ã,ã¼ã,ä,äƒng,å,ï,ï¾ÿc,ï¾ÿf
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df2['label'] = tweets_less['label'].values
df2

,,aaaaag,aaaaahhh,aaaaahhhh,aaaall,aaaand,aaaawwwhey,aaah,aaajaayradd,aaargh,...,â,ã,ã¼ã,ä,äƒng,å,ï,ï¾ÿc,ï¾ÿf,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9997,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9998,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
#Libraries installing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Dividing into training and testing sets

In [17]:
X = pd.DataFrame(df2.iloc[:,1:])
y = pd.DataFrame(df2.iloc[:,0])
X.shape

(10000, 15989)

In [18]:
y.shape

(10000, 1)

In [19]:
df2['label'].value_counts()

1    5001
0    4999
Name: label, dtype: int64

In [20]:
#spliting into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
#random state is used when data is collected form different sources, so trainin and testing will be randomly chosen
#also, you can give any number to randm state, to make sure, the data taken is same eveytime, we took 1
model2 = LogisticRegression()
model2.fit(X_train, y_train.values.ravel())

c:\users\nikhil jain\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

# Logistic Regression

In [21]:
print("Train Accuracy is", model2.score(X_train, y_train))
print("Test Accuracy is", model2.score(X_test, y_test))

Train Accuracy is 0.950625
Test Accuracy is 0.763


# Multinomial NB

In [37]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train.values.ravel())
print("Train Accuracy is", nb.score(X_train, y_train))
print("Test Accuracy is", nb.score(X_test, y_test))

Train Accuracy is 0.93325
Test Accuracy is 0.7595


# Random Forest

In [38]:
#https://stackoverflow.com/questions/34165731/a-column-vector-y-was-passed-when-a-1d-array-was-expected
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train.values.ravel())
print("Train Accuracy is", rf.score(X_train, y_train))
print("Test Accuracy is", rf.score(X_test, y_test))

Train Accuracy is 0.99975
Test Accuracy is 0.728


In [39]:
tfidf = TfidfVectorizer(analyzer = clean_text)

P = tfidf.fit_transform(tweets_less['tweet'])
print(P.shape) 

(10000, 15989)


In [40]:
print(tfidf.get_feature_names())

['', 'aaaaag', 'aaaaahhh', 'aaaaahhhh', 'aaaall', 'aaaand', 'aaaawwwhey', 'aaah', 'aaajaayradd', 'aaargh', 'aaarrrrgghhhh', 'aaaww', 'aaawwww', 'aad', 'aahoogendoorn', 'aaminahh', 'aampj', 'aargh', 'aaronfilmmak', 'aaronmiciah', 'aaronshal', 'aaru', 'abacab', 'abay', 'abbi', 'abbyjeann', 'abc', 'abcdefglyni', 'abcess', 'abdomen', 'abegrub', 'aber', 'aberdeenfc', 'aberystwythart', 'abhisheknagar', 'abhishektiwari', 'abhishekupadhya', 'abi', 'abigail', 'abigaillovesy', 'abik', 'abil', 'abipip', 'abisignorelli', 'abit', 'abl', 'ablt', 'abolish', 'aboogi', 'abroad', 'absolut', 'absolutley', 'absorb', 'abt', 'abus', 'ac', 'acabo', 'academi', 'accent', 'accept', 'access', 'accessquot', 'accid', 'accident', 'accom', 'accomod', 'accompani', 'accomplish', 'accord', 'account', 'accountlook', 'accross', 'accshan', 'acct', 'accur', 'accus', 'ace', 'acebowman', 'ach', 'achaha', 'achen', 'achetã', 'achiev', 'achievin', 'achil', 'achm', 'ack', 'ackin', 'acknowledg', 'acland', 'acm', 'acmast', 'acowbo

In [41]:
df2 = pd.DataFrame(P.toarray(), columns = tfidf.get_feature_names())
df2.head(10)

,,aaaaag,aaaaahhh,aaaaahhhh,aaaall,aaaand,aaaawwwhey,aaah,aaajaayradd,aaargh,...,á,â,ã,ã¼ã,ä,äƒng,å,ï,ï¾ÿc,ï¾ÿf
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.049242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.052199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.037961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.054089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.113833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.049790,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.098652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df2['label'] = tweets_less['label'].values
df2

,,aaaaag,aaaaahhh,aaaaahhhh,aaaall,aaaand,aaaawwwhey,aaah,aaajaayradd,aaargh,...,â,ã,ã¼ã,ä,äƒng,å,ï,ï¾ÿc,ï¾ÿf,label
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.049242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.052199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.037961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.054089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.041095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9996,0.058907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9997,0.099614,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9998,0.050544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [43]:
X = pd.DataFrame(df2.iloc[:,:-1])
y = pd.DataFrame(df2.iloc[:,-1])
X.shape

(10000, 15989)

In [44]:
y.shape

(10000, 1)

In [45]:
df2['label'].value_counts()

1    5001
0    4999
Name: label, dtype: int64

# Logistic regression

In [46]:
#spliting into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
#random state is used when data is collected form different sources, so trainin and testing will be randomly chosen
#also, you can give any number to randm state, to make sure, the data taken is same eveytime, we took 1
model = LogisticRegression()
#https://stackoverflow.com/questions/34165731/a-column-vector-y-was-passed-when-a-1d-array-was-expected
model.fit(X_train, y_train.values.ravel())

LogisticRegression()

In [47]:
print("Train Accuracy is", model.score(X_train, y_train))
print("Test Accuracy is", model.score(X_test, y_test))

Train Accuracy is 0.8955
Test Accuracy is 0.7515


# Multinomial NB

In [48]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train.values.ravel())
print("Train Accuracy is", nb.score(X_train, y_train))
print("Test Accuracy is", nb.score(X_test, y_test))

Train Accuracy is 0.93325
Test Accuracy is 0.7595


# Random Forest

In [49]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train.values.ravel())
print("Train Accuracy is", rf.score(X_train, y_train))
print("Test Accuracy is", rf.score(X_test, y_test))

Train Accuracy is 0.99975
Test Accuracy is 0.7345


In [50]:
import tweepy

In [51]:
access_token = '740071211523264512-a4CDoXv6L3ZP8v41QyLnWo2rTW0Rum7'
access_token_secret = 'Eu3qrJNGL3ocGZNOt7dm3bG4LvwjBxNfLeIsaveqPCmRT'

api_key = '6CjV0v3Fl63O1uhBKiUbuHUux'
api_secret_key = 'rlRxuUl5nBRAG7hY3wAf1NeCQgNwa3bGtJgV9fgQDoZDl2AQOx'

In [52]:
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)

In [53]:
api = tweepy.API(auth)

In [144]:
public_tweets = api.search(q = 'reliance')
df = pd.DataFrame(columns=['special_tweets'])
for tweet in public_tweets:
    df = df.append({'special_tweets': tweet.text}, ignore_index=True)
df

,special_tweets
0,RT @amandaleighlaw: The police state wasn’t built in 1 day or imagined by 1 person. It will not ...
1,"@rakz_27 Hi mam,\nCan reliance fall or not.?\nWhat can we do buy or sell?"
2,"@so_says_sally @SharonBrennan 9. And are homes already had significant issues with funding, work..."
3,RT @amandaleighlaw: The police state wasn’t built in 1 day or imagined by 1 person. It will not ...
4,RT @MylesMcNulty: ...such a huge amount of funding for it completely removes any reliance on the...
5,RT @begin_invest: The Biggies 😌\n\nVisit: https://t.co/VJzCqtupVT\n\n#begininvest #BeASmartInves...
6,RT @amandaleighlaw: The police state wasn’t built in 1 day or imagined by 1 person. It will not ...
7,New article in Ethics and Information Technology where I argue that over-reliance on internal me...
8,"RT @krishnalubb: In a nutshell, it's a great one indeed...✌️😊🙏superb and wonderful indeed...\nA ..."
9,"Amid COVID-19, Investors Bet On Mukesh Ambani's Jio And Its Giant-Killer Playbook https://t.co/i..."


In [145]:
df.shape

(15, 1)

In [157]:
# def remove_punct(text):
#     text  = "".join([char for char in text if char not in string.punctuation])
#     text = re.sub('[0-9]+', '', text)
#     return text

In [156]:
# #https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
# pd.options.mode.chained_assignment = None  # default='warn'

# df['special_tweets'] = df['special_tweets'].apply(lambda x: remove_punct(x))
# df['special_tweets'][0]

In [148]:
# def tokenization(text):
#     text = re.split('\W+', text)
#     return text

# df['special_tweets'] = df['special_tweets'].apply(lambda x: tokenization(x.lower()))

In [155]:
# #removing stopword

# stopword = nltk.corpus.stopwords.words('english')

# def remove_stopwords(text):
#     text = [word for word in text if word not in stopword]
#     return text

# df['special_tweets'] = df['special_tweets'].apply(lambda x: remove_stopwords(x))
# df['special_tweets'][0]

In [152]:
# #stemming and Lammetization
# ps = nltk.PorterStemmer()

# def stemming(text):
#     text = [ps.stem(word) for word in text]
#     return text

# df['special_tweets_stem'] = df['special_tweets_stop'].apply(lambda x: stemming(x))
# df['special_tweets_stem'][0]

In [154]:
# df['special_tweets'][0]

In [153]:
# tweet = df['special_tweets'][0]
# tweet

# tweet example

In [158]:
tweet_example = " This is a good minor project"

In [160]:
print(model.predict(tfidf.transform([tweet_example]))[0])

1


In [161]:
print(rf.predict(tfidf.transform([tweet_example]))[0])

1


In [162]:
print(nb.predict(tfidf.transform([tweet_example]))[0])

1
